In [3]:
import numpy as np
from gpkit import Model, Variable, units
from gpkit.shortcuts import Var
from gpkit.tools import te_exp_minus1

In [34]:
class BreguetRange(Model):
    def setup(self):
        g = Var("g", 9.81, "m/s^2", "gravity")
        R = Var("R", "nautical_miles", "range")
        T = Var("T", "N", "Thrust")
        h_fuel = Var("h_{fuel}", 42e6, "J/kg", "fuel heating value")
        eta_0 = Var("\\eta_0", 0.4, "-", "Overall engine efficiency")
        W = Var("W", "N", "Aircraft weight")
        W_zfw = Var("W_{zfw}", "N", "Zero fuel Weight")
        W_fuel = Var("W_{fuel}", "N", "Fuel weight")
        z_bre = Var("z_{bre}", "-", "Breguet parameter")

        # Set up Model Equations
        objective = 1/R  # Maximize range
        constraints = [W >= W_zfw + W_fuel,
                       z_bre >= g*R*T/(h_fuel*eta_0*W),
                       W_fuel/W_zfw >= te_exp_minus1(z_bre, nterm=3)
                      ]
        return objective, constraints
    
    def test(self):
        self.substitutions.update({"W_{zfw}":400, "W":500, "T":15})
        self.solve(verbosity=0)

BreguetRange().test()

In [37]:
class Sizing(Model):
    def setup(self):
        Range = BreguetRange()
        
        W = Range["W"]
        W_zfw = Range["W_{zfw}"]
        g = Range["g"]
        R = Range["R"]
        T = Range["T"]
        
        m_pay = Var("m_{pay}", 5, "kg", "Payload Mass")
        W_mto = Var("W_{mto}", 500, "N", "Mass take off weight")
        W_zfwmax = Var("W_{zfwmax}", 200, "N", "Max zfw")
        T_min = Var("T_{min}", 15, "N", "Thrust Max")
        
        constraints = [T >= T_min, 
                       W_zfw >= W_zfwmax,
                       W_mto >= W + m_pay*g]
        cost = 1/R
        model = Model(cost, constraints)
        
        for subm in [Range]:
            model = model & subm
        
        return model
    
    def test(self):
        self.solve(verbosity=0)
        
Sizing().solve()

Using solver 'cvxopt'
Solving for 6 variables.
Solving took 0.0107 seconds.

Cost
----
 4.37e-05 [1/nmi] 

Free Variables
--------------
              Sizing4 |                                   
                    R : 2.288e+04  [nmi] range            
                    T : 15         [N]   Thrust           
                    W : 451        [N]   Aircraft weight  
              W_{zfw} : 200        [N]   Zero fuel Weight 
                                                          
Sizing4BreguetRange31 |                                   
                    R : 2.288e+04  [nmi] range            
                    T : 15         [N]   Thrust           
                    W : 451        [N]   Aircraft weight  
             W_{fuel} : 251        [N]   Fuel weight      
              W_{zfw} : 200        [N]   Zero fuel Weight 
              z_{bre} : 0.8231           Breguet parameter

Constants
---------
              Sizing4 |                                            
       

{'constants': {h_{fuel}_Sizing4BreguetRange31: array(42000000.0),
  \eta_0_Sizing4BreguetRange31: array(0.4),
  W_{mto}_Sizing4: array(500),
  m_{pay}_Sizing4: array(5),
  W_{zfwmax}_Sizing4: array(200),
  g_Sizing4BreguetRange31: array(9.81),
  g_Sizing4: array(9.81),
  T_{min}_Sizing4: array(15)},
 'cost': array(4.3703891897654125e-05),
 'freevariables': {z_{bre}_Sizing4BreguetRange31: array(0.8230826803669365),
  T_Sizing4BreguetRange31: array(14.999999994331095),
  W_{fuel}_Sizing4BreguetRange31: array(250.95002246706903),
  W_{zfw}_Sizing4: array(199.9999984487894),
  T_Sizing4: array(14.999999994331095),
  R_Sizing4: array(22881.25740247121),
  W_Sizing4BreguetRange31: array(450.95000426648363),
  W_{zfw}_Sizing4BreguetRange31: array(199.9999984487894),
  R_Sizing4BreguetRange31: array(22881.25740247121),
  W_Sizing4: array(450.95000426648363)},
 'localmodel': 1.41e+04*T_{min}_Sizing4*W_{mto}_Sizing4**-2.5*W_{zfwmax}_Sizing4**1.3*\eta_0_Sizing4BreguetRange31**-1*g_Sizing4**1.2*h_